In [1]:
from itertools import product

import numpy as np
import pandas as pd

from Bio.Data import CodonTable

Adapted from varalign/notebook/03_xx_Residue_Mutability.ipynb

In [2]:
## Features of the genetic code, this might be best in its own small notebook
# want to look at the structure of the code in terms of the components of dN/dS
# and also with respect to amino acid physical properties...
# look at CG content

# table with all codon SNPs, bi-directional

genetic_code = pd.Series(CodonTable.standard_dna_table.forward_table)
codon_mutations = pd.DataFrame(list(product(genetic_code, genetic_code)),
                               columns=['from_aa', 'to_aa'])
codon_mutations.index = pd.MultiIndex.from_product([genetic_code.index, genetic_code.index],
                                                   names=['from_codon', 'to_codon'])
codon_mutations.head()

from_aa to_aa
from_codon to_codon              
TTT        TTT            F     F
           TTC            F     F
           TTA            F     L
           TTG            F     L
           TCT            F     S

In [3]:
# identify snps, mnps, etc.
def count_substitutions(s1, s2):
    assert len(s1) == len(s2)
    return len(s1) - sum([a == b for a,b in zip(s1, s2)])
    

n_subs = codon_mutations.index.to_frame().apply(lambda x: count_substitutions(*x),
                                                axis=1)
codon_mutations = codon_mutations.assign(n_subs=n_subs)
codon_mutations.head()

from_aa to_aa  n_subs
from_codon to_codon                      
TTT        TTT            F     F       0
           TTC            F     F       1
           TTA            F     L       1
           TTG            F     L       1
           TCT            F     S       1

In [4]:
# now I can identify SNPs only
codon_mutations.query('n_subs == 1').head()

from_aa to_aa  n_subs
from_codon to_codon                      
TTT        TTC            F     F       1
           TTA            F     L       1
           TTG            F     L       1
           TCT            F     S       1
           TAT            F     Y       1

In [5]:
# deduplicate direction
# pairs = list(product(codon_mutations.index.tolist(), codon_mutations.index.tolist()))

def is_sorted(x):
    return list(x) == sorted(x)  # sorted returns a list


codon_mutations_dedupe = codon_mutations.loc[[is_sorted(x) for x in codon_mutations.index.tolist()]]
codon_mutations_dedupe.head()

from_aa to_aa  n_subs
from_codon to_codon                      
TTT        TTT            F     F       0
TTC        TTT            F     F       1
           TTC            F     F       0
           TTG            F     L       1
TTA        TTT            L     F       1

In [6]:
# Ok I think this is all possible SNPs
# snps = codon_mutations_dedupe.query('n_subs == 1')
# actually, don't think I want it deduped...
snps = codon_mutations.query('n_subs == 1')
snps.to_csv('snp_table.csv')
snps

from_aa to_aa  n_subs
from_codon to_codon                      
TTT        TTC            F     F       1
           TTA            F     L       1
           TTG            F     L       1
           TCT            F     S       1
           TAT            F     Y       1
...                     ...   ...     ...
GGG        GCG            G     A       1
           GAG            G     E       1
           GGT            G     G       1
           GGC            G     G       1
           GGA            G     G       1

[526 rows x 3 columns]

There are fewer than 9 mutations for some codons because I am ignoring stops
and starts.

In [16]:
snps.groupby('from_codon').size().value_counts()

9    43
8    13
7     5
dtype: int64

In [7]:
n_snps_to_mutation = pd.crosstab(snps['from_aa'], snps['to_aa'])
n_snps_to_mutation

to_aa,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
from_aa,,,,,,,,,,,,,,,,,,,,
A,12,0,2,2,0,4,0,0,0,0,0,0,4,0,0,4,4,4,0,0
C,0,2,0,0,2,2,0,0,0,0,0,0,0,0,2,4,0,0,2,2
D,2,0,2,4,0,2,2,0,0,0,0,2,0,0,0,0,0,2,0,2
E,2,0,4,2,0,2,0,0,2,0,0,0,0,2,0,0,0,2,0,0
F,0,2,0,0,2,0,0,2,0,6,0,0,0,0,0,2,0,2,0,2
G,4,2,2,2,0,12,0,0,0,0,0,0,0,0,6,2,0,4,1,0
H,0,0,2,0,0,0,2,0,0,2,0,2,2,4,2,0,0,0,0,2
I,0,0,0,0,2,0,0,6,1,4,3,2,0,0,1,2,3,3,0,0
K,0,0,0,2,0,0,0,1,2,0,1,4,0,2,2,0,2,0,0,0


In [8]:
m = pd.crosstab(snps['from_aa'], snps['to_aa'])
# m.where(np.tril(m, -1) > 0).fillna(0)
# n possible missense, all codons
n_mis_all_codons = (m.sum(axis=1) - np.diag(m)).sort_values()
n_mis_all_codons

from_aa
W     7
M     9
Y    12
C    14
E    14
K    14
Q    14
D    16
F    16
H    16
N    16
I    21
G    23
V    24
T    24
A    24
P    24
L    33
R    34
S    37
dtype: int64

In [9]:
# n possible syn, all codons
n_syn_all_codons = pd.Series(np.diag(m), m.index).sort_values()
n_syn_all_codons

from_aa
W     0
M     0
Y     2
C     2
D     2
E     2
F     2
H     2
K     2
N     2
Q     2
I     6
V    12
T    12
A    12
P    12
G    12
S    14
R    18
L    18
dtype: int64

In [10]:
# n codons
n_codons = genetic_code.value_counts()
n_codons

R    6
S    6
L    6
T    4
G    4
A    4
V    4
P    4
I    3
D    2
Q    2
F    2
C    2
Y    2
K    2
H    2
N    2
E    2
W    1
M    1
dtype: int64

In [11]:
n_syn_all_codons.div(n_codons).round(1).sort_values()

W    0.0
M    0.0
Y    1.0
C    1.0
D    1.0
E    1.0
F    1.0
H    1.0
K    1.0
N    1.0
Q    1.0
I    2.0
S    2.3
V    3.0
T    3.0
A    3.0
P    3.0
G    3.0
R    3.0
L    3.0
dtype: float64

In [12]:
n_mis_all_codons.div(n_codons).round(1).sort_values()

L    5.5
R    5.7
G    5.8
V    6.0
T    6.0
P    6.0
A    6.0
Y    6.0
S    6.2
I    7.0
K    7.0
W    7.0
Q    7.0
E    7.0
C    7.0
N    8.0
F    8.0
D    8.0
H    8.0
M    9.0
dtype: float64

In [13]:
# so something with ++L may have more synonymous...
# likewise ++H